In [1]:
import numpy as np

data = open('shakespeare_train.txt', 'r').read() # should be simple plain text file

In [14]:
len(data)

268330

In [4]:
# Using the trained weights 
a = np.load("char-rnn-snapshot.npz",allow_pickle=True)
Wxh = a["Wxh"] # 250 x 62
Whh = a["Whh"] # 250 x 250
Why = a["Why"] # 62 x 250
bh = a["bh"] # 250 x 1
by = a["by"] # 62 x 1

In [9]:
chars, data_size, vocab_size, char_to_ix, ix_to_char = a["chars"].tolist(), a["data_size"].tolist(), a["vocab_size"].tolist(), a["char_to_ix"].tolist(), a["ix_to_char"].tolist()
# hyperparameters
hidden_size = 250
seq_length = 1000 # number of steps to unroll the RNN for

In [11]:
def sample(h, seed_ix, n, alpha):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """

  # Start Your code
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(alpha * y) / np.sum(np.exp(alpha * y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes
  # End your code

In [42]:
# Part 1: Generating Samples
def temp(length, alpha=1):
  """
  generate a sample text with assigned alpha value for temperture.
  生成一个示例文本，并为温度指定 alpha 值。
  
  """
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  inputs = [char_to_ix[ch] for ch in data[:seq_length]] # 输入的单词序列，进行独热编码，是一个序列，大小为1000*1
  # print("inputs= ",inputs,"inputs.size= ",len(inputs))
  hs = np.zeros((hidden_size,1)) # 隐藏层初始状态为0，大小为1000*1
  
  # generates a sample
  sample_ix = sample(hs, inputs[0], length, alpha) # 根据输入的inputs生成后续的length个单词
  # print("inputs= ",inputs,"inputs.size= ",len(inputs))
  # print("sample_ix= ",sample_ix,"sample_ix.size= ",len(sample_ix))
  # txt = ''.join(ix_to_char[ix] for ix in inputs)
  # print ('----\n%s \n----' % (txt, ))
  print("---------------------------------------")
  txt = ''.join(ix_to_char[ix] for ix in sample_ix)
  print ('----\n%s \n----' % (txt, )) # 输出生成的单词

In [68]:
# Part 2: Complete a String
def comp(m, n):
  """
  given a string with length m, complete the string with length n more characters
  给定一个长度为 m 的字符串，完成长度为 n 个字符的字符串
  """
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  np.random.seed()
  # the context string starts from a random position in the data
  start_index = np.random.randint(265000)
  inputs = [char_to_ix[ch] for ch in data[start_index : start_index+seq_length]] # 随机读取输入字符串中的seq_length个单词
  # txt = ''.join(ix_to_char[ix] for ix in ixes)
  # print('Context: \n----\n%s \n----\n\n\n' % (txt,))
  # print("inputs= ",inputs,"\n","inputs.size= ",len(inputs))
  h = np.zeros((hidden_size,1))
  x = np.zeros((vocab_size, 1))
  word_index = 0
  ix = inputs[word_index]
  x[ix] = 1

  ixes = []
  ixes.append(ix)

  # generates the context text
  for t in range(m):

    # Start Your code
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    # x is one of k encoding of which index is 1 in the char
    x = np.zeros((vocab_size, 1))
    ix = inputs[word_index + 1]
    word_index += 1
    x[ix] = 1
    # End your code

    ixes.append(ix)
  print(inputs)
  print(ixes)
  txt = ''.join(ix_to_char[ix] for ix in ixes)
  print('Context: \n----\n%s \n----\n\n\n' % (txt,))

  # compute the softmax probability and sample from the data 从数据中计算 softmax 概率和样本
  # and use the output as the next input where we start the continuation 并使用输出作为我们开始延续的下一个输入

  # Start Your code
  y = np.dot(Why, h) + by
  p = np.exp(y) / np.sum(np.exp(y))
  ix = np.random.choice(range(vocab_size), p=p.ravel())
  x = np.zeros((vocab_size, 1))
  x[ix] = 1
  # End your code

  # start completing the string
  ixes = []
  for t in range(n):

      # Start Your code
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
      # End your code

    ixes.append(ix)

  # generates the continuation of the string
  txt = ''.join(ix_to_char[ix] for ix in ixes)
  print('Continuation: \n----\n%s \n----' % (txt,))

In [69]:
if __name__ == '__main__':
    ### Test case
    ## Part 1
    #temp(length=200, alpha=5)
    # temp(length=200, alpha=1)
    # temp(length=200, alpha=0.1)

    # ## Part 2
    # comp(780,200)
    comp(50,500)
    # comp(2,500)
    # comp(300,300)
    # comp(100,500)

[59, 49, 55, 54, 2, 47, 49, 56, 44, 39, 54, 6, 2, 56, 49, 2, 39, 50, 56, 54, 39, 36, 56, 2, 49, 42, 2, 44, 39, 54, 0, 31, 49, 2, 47, 39, 39, 56, 2, 59, 49, 55, 2, 36, 56, 2, 56, 44, 39, 2, 31, 49, 57, 39, 54, 2, 36, 50, 40, 2, 57, 39, 48, 37, 49, 47, 39, 2, 59, 49, 55, 7, 0, 0, 34, 24, 29, 20, 9, 0, 32, 44, 36, 56, 6, 2, 57, 43, 48, 48, 2, 59, 49, 55, 2, 41, 49, 2, 55, 50, 56, 49, 2, 56, 44, 39, 2, 31, 49, 57, 39, 54, 6, 2, 47, 59, 2, 48, 49, 54, 40, 10, 0, 0, 27, 29, 18, 25, 12, 14, 2, 14, 15, 32, 11, 29, 15, 9, 0, 22, 59, 2, 48, 49, 54, 40, 2, 52, 54, 49, 56, 39, 37, 56, 49, 54, 2, 50, 39, 39, 40, 53, 2, 57, 43, 48, 48, 2, 44, 36, 58, 39, 2, 43, 56, 2, 53, 49, 7, 0, 0, 34, 24, 29, 20, 9, 0, 18, 2, 53, 44, 36, 48, 48, 2, 50, 49, 56, 2, 53, 48, 39, 39, 52, 2, 43, 50, 2, 51, 55, 43, 39, 56, 2, 36, 56, 2, 56, 44, 39, 2, 31, 49, 57, 39, 54, 7, 0, 0, 16, 23, 24, 30, 12, 14, 28, 31, 14, 29, 9, 0, 32, 44, 59, 6, 2, 57, 44, 36, 56, 2, 53, 44, 49, 55, 48, 40, 2, 59, 49, 55, 2, 42, 39, 36, 54, 